***Extração de Dados***

 *API de Clima:* https://openweathermap.org/api
 
 *API de Trânsito:* https://developers.google.com/maps/documentation/directions/overview

***Importação e Configuração Iniciais***

In [1]:
import os
import pandas as pd
import requests
import json
import sqlite3
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

***Funções de Extração de Dados***

In [2]:
import os
import requests
import pandas as pd
from IPython.display import display

def get_hourly_forecast(api_key, city):
    """Extração de previsão meteorológica de hora em hora para uma cidade."""
    try:
        url = f"http://api.openweathermap.org/data/2.5/forecast?appid={api_key}&q={city}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()['list'][:5]  # Obtém apenas as previsões das próximas 5 horas para simplificação
        return pd.json_normalize(data)
    except requests.RequestException as e:
        print(f"Erro ao acessar a API de Previsão de Clima: {e}")
        return pd.DataFrame()

def get_traffic_data(api_key, origin, destination):
    """Extração de dados de trânsito com verificações apropriadas."""
    try:
        url = f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&key={api_key}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if 'routes' in data and data['routes'] and 'legs' in data['routes'][0]:
            steps = data['routes'][0]['legs'][0]['steps']
            return pd.json_normalize(steps)
        else:
            print("Nenhuma rota encontrada ou dados insuficientes na resposta.")
            return pd.DataFrame()
    except requests.RequestException as e:
        print(f"Erro ao acessar a API de Trânsito: {e}")
        return pd.DataFrame()


***Chamada de Extração de Dados e Visualização***

In [3]:
# Utilizando as variáveis de ambiente para as chaves de API
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')
TRAFFIC_API_KEY = os.getenv('TRAFFIC_API_KEY')

# Definindo valores válidos para as localizações de origem e destino
valid_origin = "Avenida Paulista, São Paulo"
valid_destination = "Parque Ibirapuera, São Paulo"

# Chamadas das funções de extração
weather_df = get_hourly_forecast(WEATHER_API_KEY, 'São Paulo')
traffic_df = get_traffic_data(TRAFFIC_API_KEY, valid_origin, valid_destination)

# Visualizando o cabeçalho e os primeiros 5 registros do DataFrame Meteorológico
print("DataFrame Meteorológico - Cabeçalho e Primeiros 5 Registros:")
display(weather_df.head())

# Visualizando o cabeçalho e os primeiros 5 registros do DataFrame de Trânsito
print("DataFrame de Trânsito - Cabeçalho e Primeiros 5 Registros:")
display(traffic_df.head())



DataFrame Meteorológico - Cabeçalho e Primeiros 5 Registros:


,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.sea_level,main.grnd_level,main.humidity,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,sys.pod
0,1718323200,"[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]",10000,0,2024-06-14 00:00:00,294.36,293.82,294.09,294.36,1020,1020,929,49,0.27,0,2.62,7,10.80,n
1,1718334000,"[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]",10000,0,2024-06-14 03:00:00,292.93,292.19,292.15,292.93,1020,1020,929,47,0.78,0,3.05,14,10.38,n
2,1718344800,"[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]",10000,0,2024-06-14 06:00:00,290.64,289.70,290.64,290.64,1018,1018,928,48,0.00,0,2.91,12,12.18,n
3,1718355600,"[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]",10000,0,2024-06-14 09:00:00,289.51,288.51,289.51,289.51,1019,1019,929,50,0.00,0,2.80,19,10.54,n
4,1718366400,"[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]",10000,0,2024-06-14 12:00:00,292.54,291.61,292.54,292.54,1021,1021,931,41,0.00,0,3.02,17,7.16,d


DataFrame de Trânsito - Cabeçalho e Primeiros 5 Registros:


,html_instructions,travel_mode,distance.text,distance.value,duration.text,duration.value,end_location.lat,end_location.lng,polyline.points,start_location.lat,start_location.lng,maneuver
0,"Head <b>northwest</b> on <b>Av. Paulista</b><div style=""font-size:0.9em"">Closed Sundays 10:00 AM – 7:00 PM</div><div style=""font-size:0.9em"">Pass by Drogaria São Paulo (on the right)</div>",DRIVING,0.3 km,345,2 mins,93,-23.559178,-46.658909,lyxnC`rw{G]`@EDEBCFMLe@l@_@`@GFs@v@c@f@EDc@f@QR[^MLEF[^]`@i@n@,-23.561355,-46.656494,NaN
1,"Turn <b>right</b> after Banca Paulista V (on the right)<div style=""font-size:0.9em"">Pass by Carrefour Express Frei Caneca (on the left in 350m)</div>",DRIVING,0.5 km,492,2 mins,125,-23.555567,-46.656114,zkxnCdax{G[Se@[CCy@k@m@c@UQGEy@m@wB{Ac@[]WWOIGYSm@c@YSWSWSq@c@KI_@[,-23.559178,-46.658909,turn-right
2,Turn <b>right</b> onto <b>R. Barbosa Rodrigues</b>,DRIVING,78 m,78,1 min,23,-23.556041,-46.655549,huwnCtow{G`@e@HIDEV[DGBEHO,-23.555567,-46.656114,turn-right
3,Turn <b>right</b> onto <b>Rua Peixoto Gomide</b>,DRIVING,0.1 km,119,1 min,30,-23.557111,-46.655506,fxwnCdlw{GTBJ?PAf@Ah@AF?T?PAJAF?JA,-23.556041,-46.655549,turn-right
4,Turn <b>left</b> onto <b>R. Barata Ribeiro</b>,DRIVING,0.3 km,266,1 min,59,-23.556436,-46.653007,|~wnC|kw{GY_BIi@G_@AGCKQ}@QcAQcACMEWSiA,-23.557111,-46.655506,turn-left


**Limpeza e Transformação:**

*° Remoção de duplicatas e registros inválidos.* 

*° Padronização de formatos (datas, números, etc.).*

*° Tratamento de valores nulos ou ausentes.*

*° Identificação de tendências ou padrôes nos numeros sorteados*

*Funções de Limpeza, Transformação e Remoção de Duplicatas*

In [4]:
def clean_and_transform_weather_data(df):
    """Limpeza, transformação e verificação de duplicatas dos dados meteorológicos."""
    print(f"Registros antes da remoção de duplicatas: {len(df)}")
    
    # Fazendo uma cópia do DataFrame para evitar SettingWithCopyWarning
    df = df.copy()
    
    # Converte timestamp para datetime legível
    df['dt_txt'] = pd.to_datetime(df['dt_txt'])
    
    # Seleciona e renomeia colunas de interesse
    df = df[['dt_txt', 'main.temp', 'main.humidity', 'weather', 'wind.speed']]
    df.columns = ['Date_Time', 'Temperature', 'Humidity', 'Weather', 'Wind_Speed']
    
    # Converte a lista de detalhes do tempo em string de descrição usando loc para evitar warning
    df.loc[:, 'Weather'] = df['Weather'].apply(lambda x: x[0]['description'] if x else None)
    
    # Remove duplicatas
    df = df.drop_duplicates()
    
    print(f"Registros após a remoção de duplicatas: {len(df)}")
    
    return df

def clean_and_transform_traffic_data(df):
    """Limpeza, transformação e verificação de duplicatas dos dados de trânsito."""
    print(f"Registros antes da remoção de duplicatas: {len(df)}")
    
    df = df.copy()  # Cópia do DataFrame para manipulação segura
    
    # Seleciona e renomeia colunas de interesse
    df = df[['duration.text', 'start_location.lat', 'start_location.lng', 'end_location.lat', 'end_location.lng']]
    df.columns = ['Duration', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']
    
    # Remove duplicatas
    df = df.drop_duplicates()
    
    print(f"Registros após a remoção de duplicatas: {len(df)}")
    
    return df


In [5]:
# Aplicando funções de limpeza e transformação
cleaned_weather_df = clean_and_transform_weather_data(weather_df)
cleaned_traffic_df = clean_and_transform_traffic_data(traffic_df)

# Visualizando os dataframes limpos e transformados
print("Dados Meteorológicos Limpos e Transformados:")
display(cleaned_weather_df.head())

print("Dados de Trânsito Limpos e Transformados:")
display(cleaned_traffic_df.head())


Registros antes da remoção de duplicatas: 5
Registros após a remoção de duplicatas: 5
Registros antes da remoção de duplicatas: 13
Registros após a remoção de duplicatas: 13
Dados Meteorológicos Limpos e Transformados:


,Date_Time,Temperature,Humidity,Weather,Wind_Speed
0,2024-06-14 00:00:00,294.36,49,clear sky,2.62
1,2024-06-14 03:00:00,292.93,47,clear sky,3.05
2,2024-06-14 06:00:00,290.64,48,clear sky,2.91
3,2024-06-14 09:00:00,289.51,50,clear sky,2.80
4,2024-06-14 12:00:00,292.54,41,clear sky,3.02


Dados de Trânsito Limpos e Transformados:


,Duration,Start_Lat,Start_Lng,End_Lat,End_Lng
0,2 mins,-23.561355,-46.656494,-23.559178,-46.658909
1,2 mins,-23.559178,-46.658909,-23.555567,-46.656114
2,1 min,-23.555567,-46.656114,-23.556041,-46.655549
3,1 min,-23.556041,-46.655549,-23.557111,-46.655506
4,1 min,-23.557111,-46.655506,-23.556436,-46.653007


**Verificando se todas as duplicatas foram excluidas**
 - Contagem de registros antes e depois da remoção das duplicatas.
 - Verificar se ainda existem registros duplicados após a remoção.

**Extração e Contagem de Registros de Trânsito antes e depois da remoção das duplicatas**
- Dados Meológicos e Trânsito

In [6]:
# Função para extrair dados meteorológicos
def get_weather_data(api_key, city):
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = f"{base_url}q={city}&appid={api_key}"
    response = requests.get(complete_url)
    return response.json()

# Função para extrair dados de trânsito
def get_traffic_data(api_key, origin, destination):
    base_url = "https://maps.googleapis.com/maps/api/directions/json?"
    complete_url = f"{base_url}origin={origin}&destination={destination}&key={api_key}"
    response = requests.get(complete_url)
    return response.json()

# API keys
weather_api_key = os.getenv('WEATHER_API_KEY')
traffic_api_key = os.getenv('TRAFFIC_API_KEY')

# Obtendo dados meteorológicos
weather_data = get_weather_data(weather_api_key, "São Paulo")
weather_df = pd.json_normalize(weather_data)

# Adicionando linhas duplicadas manualmente para teste
weather_df = pd.concat([weather_df]*5, ignore_index=True)

# Contagem de registros antes da remoção de duplicatas
initial_count_weather = len(weather_df)
print(f"Contagem inicial de registros meteorológicos: {initial_count_weather}")

# Convertendo colunas com listas para strings
for col in weather_df.columns:
    if isinstance(weather_df[col].iloc[0], list):
        weather_df[col] = weather_df[col].apply(str)

# Remoção de duplicatas nos dados meteorológicos
weather_df = weather_df.drop_duplicates()

# Contagem de registros após a remoção de duplicatas
final_count_weather = len(weather_df)
print(f"Contagem final de registros meteorológicos: {final_count_weather}")

# Obtendo dados de trânsito
traffic_data = get_traffic_data(traffic_api_key, "New York, NY", "Los Angeles, CA")

# Extraindo detalhes das etapas da rota, se disponível
if 'routes' in traffic_data and traffic_data['routes']:
    steps = traffic_data['routes'][0]['legs'][0]['steps']
    traffic_df = pd.json_normalize(steps)
else:
    traffic_df = pd.DataFrame()  # Cria um DataFrame vazio se não houver dados de trânsito

# Adicionando linhas duplicadas manualmente para teste
traffic_df = pd.concat([traffic_df]*5, ignore_index=True)

# Contagem de registros antes da remoção de duplicatas
initial_count_traffic = len(traffic_df)
print(f"Contagem inicial de registros de trânsito: {initial_count_traffic}")

# Convertendo colunas com listas para strings
for col in traffic_df.columns:
    if isinstance(traffic_df[col].iloc[0], list):
        traffic_df[col] = traffic_df[col].apply(str)

# Remoção de duplicatas nos dados de trânsito
traffic_df = traffic_df.drop_duplicates()

# Contagem de registros após a remoção de duplicatas
final_count_traffic = len(traffic_df)
print(f"Contagem final de registros de trânsito: {final_count_traffic}")

Contagem inicial de registros meteorológicos: 5
Contagem final de registros meteorológicos: 1
Contagem inicial de registros de trânsito: 255
Contagem final de registros de trânsito: 51


In [7]:
remaining_duplicates_weather = weather_df[weather_df.duplicated()]

# Verificação de duplicatas restantes nos dados de trânsito
remaining_duplicates_traffic = traffic_df[traffic_df.duplicated()]

print("Verificação de duplicatas restantes nos dados meteorológicos:")
display(remaining_duplicates_weather)

print("Verificação de duplicatas restantes nos dados de trânsito:")
display(remaining_duplicates_traffic)


Verificação de duplicatas restantes nos dados meteorológicos:


,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset


Verificação de duplicatas restantes nos dados de trânsito:


,html_instructions,travel_mode,distance.text,distance.value,duration.text,duration.value,end_location.lat,end_location.lng,polyline.points,start_location.lat,start_location.lng,maneuver


**Padronização de formatos (datas, números, etc.).**

In [8]:
import pandas as pd
from IPython.display import display

# Função para garantir um número mínimo de registros
def ensure_minimum_records(df, min_records):
    if len(df) < min_records:
        df = pd.concat([df] + [df.iloc[[0]]] * (min_records - len(df)), ignore_index=True)
    return df

# Padronização de formatos nos dados meteorológicos
def padronizar_dados_meteorologicos(df):
    df = df.copy()
    # Convertendo datas para datetime e formatando
    df['dt'] = pd.to_datetime(df['dt'], unit='s', errors='coerce').dt.strftime('%d/%m/%Y')
    df['sys.sunrise'] = pd.to_datetime(df['sys.sunrise'], unit='s', errors='coerce').dt.strftime('%d/%m/%Y')
    df['sys.sunset'] = pd.to_datetime(df['sys.sunset'], unit='s', errors='coerce').dt.strftime('%d/%m/%Y')

    # Preenchendo valores nulos com 'Data Inválida'
    df['dt'].fillna('Data Inválida', inplace=True)
    df['sys.sunrise'].fillna('Data Inválida', inplace=True)
    df['sys.sunset'].fillna('Data Inválida', inplace=True)

    # Convertendo números para floats/inteiros apropriados
    numeric_cols = ['visibility', 'timezone', 'main.temp', 'main.feels_like', 'main.temp_min', 'main.temp_max', 'main.pressure', 'main.humidity', 'wind.speed', 'wind.deg', 'clouds.all', 'sys.type', 'sys.id']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    df[numeric_cols] = df[numeric_cols].fillna(0)
    return df

# Padronização de formatos nos dados de trânsito
def padronizar_dados_transito(df):
    df = df.copy()
    # Convertendo números para floats/inteiros apropriados
    numeric_cols = ['distance.value', 'duration.value', 'end_location.lat', 'end_location.lng', 'start_location.lat', 'start_location.lng']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    df[numeric_cols] = df[numeric_cols].fillna(0)

    # Preenchendo valores nulos com strings apropriadas
    df['html_instructions'].fillna('Instrução Inválida', inplace=True)
    df['travel_mode'].fillna('Modo Inválido', inplace=True)
    return df

# Garantindo registros mínimos
weather_df = ensure_minimum_records(weather_df, 5)
traffic_df = ensure_minimum_records(traffic_df, 5)

# Padronização de dados
weather_df = padronizar_dados_meteorologicos(weather_df)
traffic_df = padronizar_dados_transito(traffic_df)

# Exibindo os primeiros 5 registros após padronização
print("Dados meteorológicos após padronização:")
display(weather_df.head(5))
print("Dados de trânsito após padronização:")
display(traffic_df.head(5))


Dados meteorológicos após padronização:


,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]",stations,10000,13/06/2024,-10800,3448439,São Paulo,200,-46.6361,-23.5475,294.53,294.08,292.09,296.35,1021,52,3.6,340,0,1,8394,BR,13/06/2024,13/06/2024
1,"[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]",stations,10000,13/06/2024,-10800,3448439,São Paulo,200,-46.6361,-23.5475,294.53,294.08,292.09,296.35,1021,52,3.6,340,0,1,8394,BR,13/06/2024,13/06/2024
2,"[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]",stations,10000,13/06/2024,-10800,3448439,São Paulo,200,-46.6361,-23.5475,294.53,294.08,292.09,296.35,1021,52,3.6,340,0,1,8394,BR,13/06/2024,13/06/2024
3,"[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]",stations,10000,13/06/2024,-10800,3448439,São Paulo,200,-46.6361,-23.5475,294.53,294.08,292.09,296.35,1021,52,3.6,340,0,1,8394,BR,13/06/2024,13/06/2024
4,"[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]",stations,10000,13/06/2024,-10800,3448439,São Paulo,200,-46.6361,-23.5475,294.53,294.08,292.09,296.35,1021,52,3.6,340,0,1,8394,BR,13/06/2024,13/06/2024


Dados de trânsito após padronização:


,html_instructions,travel_mode,distance.text,distance.value,duration.text,duration.value,end_location.lat,end_location.lng,polyline.points,start_location.lat,start_location.lng,maneuver
0,"Head <b>northwest</b> toward <b>Murray St</b><div style=""font-size:0.9em"">Partial restricted usage road</div>",DRIVING,230 ft,70,1 min,15,40.713043,-74.007256,gunwFblubMGLm@~AQX,40.712680,-74.006576,NaN
1,Continue onto <b>Murray St</b>,DRIVING,0.1 mi,158,1 min,45,40.713804,-74.008826,ownwFjpubMONA@eCfH,40.713043,-74.007256,NaN
2,Turn <b>right</b> onto <b>Church St</b>,DRIVING,0.3 mi,539,2 mins,148,40.717971,-74.005553,g|nwFdzubMg@_@s@g@SOcAu@m@e@s@g@o@e@}@o@s@g@_As@i@a@oBsAQMi@a@k@a@OMGE[UIGIG,40.713804,-74.008826,turn-right
3,Keep <b>left</b> to continue on <b>6th Ave</b>/<wbr/><b>Ave of the Americas</b>,DRIVING,0.4 mi,644,3 mins,159,40.723641,-74.004797,ivowFteubMIEEAKGMGKGGCECEAGCEAg@IKCMCOCKAKCMAI?K@q@?_BBS?aABeA@qADcADQ@S@[BWDS@_@@G?I@I?SGgCw@}Bs@,40.717971,-74.005553,keep-left
4,Turn <b>left</b> onto <b>Watts St</b>,DRIVING,341 ft,104,1 min,38,40.723765,-74.006022,wypwF~`ubMARAPAJCl@C^Cv@Gz@,40.723641,-74.004797,turn-left


***Padronização de Dados Meteorológicos:***

- Datas: Convertidas para o formato dd/mm/aaaa.
- Valores Numéricos: Convertidos para tipos numéricos apropriados.
- Valores Nulos: Preenchidos com "Data Inválida".
- dt, sys.sunrise, sys.sunset preenchidos com "Data Inválida" quando necessário.
- Confirmar que os dados numéricos estão corretamente convertidos.

**Dados de Trânsito:**

- HTML: Limpo das instruções para melhorar a legibilidade.
- Polilinhas: Decodificadas em coordenadas latitude/longitude.
- Maneuver: Valores nulos substituídos por "none".
- maneuver preenchidos com "none" onde havia nulos.
- Adicionada uma verificação para substituir valores None na coluna decoded_polyline por uma lista vazia [].


In [9]:
import pandas as pd
import polyline
from IPython.display import display

# Dados simulados para trânsito
traffic_df = pd.DataFrame({
    'html_instructions': [
        "Head <b>northwest</b> toward <b>Murray St</b><div style='font-size:0.9em'>Partial restricted usage road</div>",
        "Continue onto <b>Murray St</b>",
        "Turn <b>right</b> onto <b>Church St</b>",
        "Continue onto <b>Church St</b>",
        "Turn <b>left</b> onto <b>Barclay St</b>"
    ] * 5,
    'travel_mode': ['DRIVING'] * 25,
    'distance.text': ['230 ft', '0.1 mi', '0.1 mi', '0.2 mi', '0.3 mi'] * 5,
    'distance.value': [70, 160, 300, 320, 480] * 5,
    'duration.text': ['1 min', '2 mins', '3 mins', '4 mins', '5 mins'] * 5,
    'duration.value': [60, 120, 180, 240, 300] * 5,
    'end_location.lat': [40.713043, 40.713804, 40.715428, 40.717843, 40.719678] * 5,
    'end_location.lng': [-74.007256, -74.008826, -74.007540, -74.005461, -74.003117] * 5,
    'polyline.points': ['gunwFblubMGJm@`BQX', 'ownwFjpubMONA@eCfH', 'abcdEfghijklMNOPQR', 'stuVwxyzABCD', 'efghIjklMNOPQ'] * 5,
    'start_location.lat': [40.712680, 40.713043, 40.713804, 40.715428, 40.717843] * 5,
    'start_location.lng': [-74.006576, -74.007256, -74.008826, -74.007540, -74.005461] * 5,
    'maneuver': [None] * 25
})

# Dados simulados para meteorologia
weather_df = pd.DataFrame({
    'base': ['stations'] * 5,
    'visibility': [10000] * 5,
    'dt': ['2024-06-01', '2024-06-02', '2024-06-03', '2024-06-04', '2024-06-05'],
    'timezone': [-10800] * 5,
    'id': [3448439] * 5,
    'name': ['São Paulo'] * 5,
    'cod': [200] * 5,
    'coord.lon': [-46.6361] * 5,
    'coord.lat': [-23.5475] * 5,
    'main.temp': [300.23, 300.23, 300.23, 300.23, 300.23],
    'main.feels_like': [299.92] * 5,
    'main.temp_min': [299.9] * 5,
    'main.temp_max': [301.09] * 5,
    'main.pressure': [1021] * 5,
    'main.humidity': [37] * 5,
    'wind.speed': [2.68] * 5,
    'wind.deg': [8] * 5,
    'wind.gust': [3.13] * 5,
    'clouds.all': [0] * 5,
    'sys.type': [1] * 5,
    'sys.id': [8394] * 5,
    'sys.country': ['BR'] * 5,
    'sys.sunrise': ['2024-06-01', '2024-06-02', '2024-06-03', '2024-06-04', '2024-06-05'],
    'sys.sunset': ['2024-06-01', '2024-06-02', '2024-06-03', '2024-06-04', '2024-06-05'],
    'id_weather': [800] * 5,
    'main': ['Clear'] * 5,
    'description': ['clear sky'] * 5,
    'icon': ['01d'] * 5
})

# Função para padronizar dados meteorológicos
def padronizar_dados_meteorologicos(df):
    df = df.copy()
    
    # Convertendo datas para datetime
    df['dt'] = pd.to_datetime(df['dt'], errors='coerce')
    df['sys.sunrise'] = pd.to_datetime(df['sys.sunrise'], errors='coerce')
    df['sys.sunset'] = pd.to_datetime(df['sys.sunset'], errors='coerce')
    
    # Preenchendo valores nulos com 'Data Inválida'
    df['dt'] = df['dt'].fillna('Data Inválida').apply(lambda x: x if x == 'Data Inválida' else x.strftime('%d/%m/%Y'))
    df['sys.sunrise'] = df['sys.sunrise'].fillna('Data Inválida').apply(lambda x: x if x == 'Data Inválida' else x.strftime('%d/%m/%Y'))
    df['sys.sunset'] = df['sys.sunset'].fillna('Data Inválida').apply(lambda x: x if x == 'Data Inválida' else x.strftime('%d/%m/%Y'))
    
    # Convertendo números para floats/inteiros apropriados
    numeric_cols = ['visibility', 'timezone', 'main.temp', 'main.feels_like', 'main.temp_min', 'main.temp_max', 
                    'main.pressure', 'main.humidity', 'wind.speed', 'wind.deg', 'clouds.all', 'sys.type', 'sys.id']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
    
    return df

# Função para decodificar polilinhas nos dados de trânsito
def decodificar_polylines(df):
    def safe_decode(poly):
        try:
            return polyline.decode(poly)
        except Exception as e:
            return []
    df['decoded_polyline'] = df['polyline.points'].apply(safe_decode)
    return df

# Função para limpar instruções HTML
def limpar_html(df):
    df['html_instructions'] = df['html_instructions'].str.replace(r'<.*?>', '', regex=True)
    return df

# Função para tratar valores nulos em 'maneuver'
def tratar_maneuver(df):
    df['maneuver'] = df['maneuver'].fillna('no maneuver')
    return df

# Aplicando as funções de limpeza e padronização
weather_df = padronizar_dados_meteorologicos(weather_df)
traffic_df = decodificar_polylines(traffic_df)
traffic_df = limpar_html(traffic_df)
traffic_df = tratar_maneuver(traffic_df)

# Exibindo os primeiros 5 registros após padronização
print("Dados meteorológicos após padronização:")
display(weather_df.head(5))

print("Dados de trânsito após padronização:")
display(traffic_df.head(5))


Dados meteorológicos após padronização:


,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset,id_weather,main,description,icon
0,stations,10000,01/06/2024,-10800,3448439,São Paulo,200,-46.6361,-23.5475,300.23,299.92,299.9,301.09,1021,37,2.68,8,3.13,0,1,8394,BR,01/06/2024,01/06/2024,800,Clear,clear sky,01d
1,stations,10000,02/06/2024,-10800,3448439,São Paulo,200,-46.6361,-23.5475,300.23,299.92,299.9,301.09,1021,37,2.68,8,3.13,0,1,8394,BR,02/06/2024,02/06/2024,800,Clear,clear sky,01d
2,stations,10000,03/06/2024,-10800,3448439,São Paulo,200,-46.6361,-23.5475,300.23,299.92,299.9,301.09,1021,37,2.68,8,3.13,0,1,8394,BR,03/06/2024,03/06/2024,800,Clear,clear sky,01d
3,stations,10000,04/06/2024,-10800,3448439,São Paulo,200,-46.6361,-23.5475,300.23,299.92,299.9,301.09,1021,37,2.68,8,3.13,0,1,8394,BR,04/06/2024,04/06/2024,800,Clear,clear sky,01d
4,stations,10000,05/06/2024,-10800,3448439,São Paulo,200,-46.6361,-23.5475,300.23,299.92,299.9,301.09,1021,37,2.68,8,3.13,0,1,8394,BR,05/06/2024,05/06/2024,800,Clear,clear sky,01d


Dados de trânsito após padronização:


,html_instructions,travel_mode,distance.text,distance.value,duration.text,duration.value,end_location.lat,end_location.lng,polyline.points,start_location.lat,start_location.lng,maneuver,decoded_polyline
0,Head northwest toward Murray StPartial restricted usage road,DRIVING,230 ft,70,1 min,60,40.713043,-74.007256,gunwFblubMGJm@`BQX,40.712680,-74.006576,no maneuver,"[(40.71268, -74.00658), (40.71272, -74.00664), (40.71295, -74.00713), (40.71304, -74.00726)]"
1,Continue onto Murray St,DRIVING,0.1 mi,160,2 mins,120,40.713804,-74.008826,ownwFjpubMONA@eCfH,40.713043,-74.007256,no maneuver,"[(40.71304, -74.00726), (40.71312, -74.00734), (40.71313, -74.00735), (40.7138, -74.00883)]"
2,Turn right onto Church St,DRIVING,0.1 mi,300,3 mins,180,40.715428,-74.007540,abcdEfghijklMNOPQR,40.713804,-74.008826,no maneuver,[]
3,Continue onto Church St,DRIVING,0.2 mi,320,4 mins,240,40.717843,-74.005461,stuVwxyzABCD,40.715428,-74.007540,no maneuver,[]
4,Turn left onto Barclay St,DRIVING,0.3 mi,480,5 mins,300,40.719678,-74.003117,efghIjklMNOPQ,40.717843,-74.005461,no maneuver,"[(53.94547, -2.3623), (53.94539, -2.36222), (53.9453, -2.36213)]"


3. ***Modelagem de Dados:***
    - Foi criado o esquema de banco de dados que irá armazenar os dados. Isso incluirá a definição de tabelas, relacionamentos entre elas, índices e chaves primárias/estrangeiras.

In [13]:
import sqlite3

try:
    # Conectar ao banco de dados
    conn = sqlite3.connect('weather_traffic.db')
    cur = conn.cursor()

    # Excluir a tabela existente, se necessário
    cur.execute('DROP TABLE IF EXISTS traffic')

    # Tentativa de criar novamente a tabela de dados de trânsito com a estrutura correta
    cur.execute('''
    CREATE TABLE traffic (
        traffic_id INTEGER PRIMARY KEY AUTOINCREMENT,
        html_instructions TEXT,
        travel_mode TEXT,
        distance_text TEXT,
        distance_value INTEGER,
        duration_text TEXT,
        duration_value INTEGER,
        end_location_lat REAL,
        end_location_lng REAL,
        polyline_points TEXT,
        start_location_lat REAL,
        start_location_lng REAL,
        maneuver TEXT,
        decoded_polyline TEXT
    )
    ''')

    # Verificar se a tabela foi criada com sucesso
    cur.execute("PRAGMA table_info(traffic)")
    columns_info = cur.fetchall()

    if not columns_info:
        print("A tabela não foi criada.")
    else:
        for info in columns_info:
            print(info)

except Exception as e:
    print(f"Erro ao criar a tabela: {e}")
finally:
    conn.close()


(0, 'traffic_id', 'INTEGER', 0, None, 1)
(1, 'html_instructions', 'TEXT', 0, None, 0)
(2, 'travel_mode', 'TEXT', 0, None, 0)
(3, 'distance_text', 'TEXT', 0, None, 0)
(4, 'distance_value', 'INTEGER', 0, None, 0)
(5, 'duration_text', 'TEXT', 0, None, 0)
(6, 'duration_value', 'INTEGER', 0, None, 0)
(7, 'end_location_lat', 'REAL', 0, None, 0)
(8, 'end_location_lng', 'REAL', 0, None, 0)
(9, 'polyline_points', 'TEXT', 0, None, 0)
(10, 'start_location_lat', 'REAL', 0, None, 0)
(11, 'start_location_lng', 'REAL', 0, None, 0)
(12, 'maneuver', 'TEXT', 0, None, 0)
(13, 'decoded_polyline', 'TEXT', 0, None, 0)


In [14]:
import sqlite3

def check_table_structure(conn, table_name):
    # Função para imprimir a estrutura de uma tabela
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table_name})")
    columns_info = cur.fetchall()
    if not columns_info:
        print(f"A tabela {table_name} não foi criada ou está vazia.")
    else:
        print(f"Estrutura da tabela {table_name}:")
        for info in columns_info:
            print(info)

try:
    # Conectar ao banco de dados
    conn = sqlite3.connect('weather_traffic.db')
    
    # Verificar a estrutura das tabelas
    check_table_structure(conn, 'city')
    check_table_structure(conn, 'weather')
    check_table_structure(conn, 'traffic')

    # Criar índices para melhorar a performance das consultas, se ainda não criados
    cur = conn.cursor()
    cur.execute('CREATE INDEX IF NOT EXISTS idx_weather_city_id ON weather(city_id)')
    cur.execute('CREATE INDEX IF NOT EXISTS idx_traffic_start_location ON traffic(start_location_lat, start_location_lng)')

    print("Verificações concluídas e índices criados.")

except Exception as e:
    print(f"Erro ao verificar as tabelas ou criar índices: {e}")
finally:
    conn.close()


Estrutura da tabela city:
(0, 'city_id', 'INTEGER', 0, None, 1)
(1, 'name', 'TEXT', 1, None, 0)
(2, 'coord_lon', 'REAL', 1, None, 0)
(3, 'coord_lat', 'REAL', 1, None, 0)
Estrutura da tabela weather:
(0, 'weather_id', 'INTEGER', 0, None, 1)
(1, 'base', 'TEXT', 0, None, 0)
(2, 'visibility', 'INTEGER', 0, None, 0)
(3, 'dt', 'TEXT', 0, None, 0)
(4, 'timezone', 'INTEGER', 0, None, 0)
(5, 'city_id', 'INTEGER', 0, None, 0)
(6, 'cod', 'INTEGER', 0, None, 0)
(7, 'main_temp', 'REAL', 0, None, 0)
(8, 'main_feels_like', 'REAL', 0, None, 0)
(9, 'main_temp_min', 'REAL', 0, None, 0)
(10, 'main_temp_max', 'REAL', 0, None, 0)
(11, 'main_pressure', 'INTEGER', 0, None, 0)
(12, 'main_humidity', 'INTEGER', 0, None, 0)
(13, 'wind_speed', 'REAL', 0, None, 0)
(14, 'wind_deg', 'INTEGER', 0, None, 0)
(15, 'wind_gust', 'REAL', 0, None, 0)
(16, 'clouds_all', 'INTEGER', 0, None, 0)
(17, 'sys_type', 'INTEGER', 0, None, 0)
(18, 'sys_id', 'INTEGER', 0, None, 0)
(19, 'sys_country', 'TEXT', 0, None, 0)
(20, 'sys_sunris

In [16]:
import sqlite3

# Conectar ao banco de dados
conn = sqlite3.connect('weather_traffic.db')
cur = conn.cursor()

# Dados de exemplo para inserir
dados_traffic = [
    (1, "Continue straight to stay on São João Ave", "DRIVING", "3 km", 3000, "6 mins", 360, -23.5489, -46.6388, "encoded_polyline_here", -23.5505, -46.6333, "straight", "decoded_polyline_here"),
    (2, "Turn right onto Ipiranga Ave", "DRIVING", "1 km", 1000, "2 mins", 120, -23.5452, -46.6418, "encoded_polyline_here", -23.5489, -46.6388, "right", "decoded_polyline_here")
]

# Inserção de dados na tabela traffic
cur.executemany('''
    INSERT INTO traffic (traffic_id, html_instructions, travel_mode, distance_text, distance_value, duration_text, duration_value, end_location_lat, end_location_lng, polyline_points, start_location_lat, start_location_lng, maneuver, decoded_polyline)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', dados_traffic)

# Commit das mudanças e fechar a conexão
conn.commit()
conn.close()



***Armazenamento dos Dados***

In [17]:
import sqlite3
import pandas as pd
from tabulate import tabulate

# Conectar ao banco de dados
conn = sqlite3.connect('weather_traffic.db')

# Função para exibir dados de uma tabela
def exibir_dados_tabela(conn, tabela):
    query = f'SELECT * FROM {tabela}'
    df = pd.read_sql_query(query, conn)
    if df.empty:
        print(f'Dados da tabela {tabela}: Vazia')
    else:
        print(f'Dados da tabela {tabela}:')
        print(tabulate(df, headers='keys', tablefmt='pretty'))

# Exibir dados das tabelas
exibir_dados_tabela(conn, 'city')
exibir_dados_tabela(conn, 'weather')
exibir_dados_tabela(conn, 'traffic')

# Fechar a conexão
conn.close()


Dados da tabela city:
+---+---------+-----------+-----------+-----------+
|   | city_id |   name    | coord_lon | coord_lat |
+---+---------+-----------+-----------+-----------+
| 0 |    1    | São Paulo | -46.6361  | -23.5475  |
+---+---------+-----------+-----------+-----------+
Dados da tabela weather:
+---+------------+----------+------------+---------------------+----------+---------+-----+-----------+-----------------+---------------+---------------+---------------+---------------+------------+----------+-----------+------------+----------+--------+-------------+---------------------+---------------------+------------+-------+-------------+------+
|   | weather_id |   base   | visibility |         dt          | timezone | city_id | cod | main_temp | main_feels_like | main_temp_min | main_temp_max | main_pressure | main_humidity | wind_speed | wind_deg | wind_gust | clouds_all | sys_type | sys_id | sys_country |     sys_sunrise     |     sys_sunset      | id_weather | main  | desc

In [23]:
import sqlite3
from datetime import datetime

def formatar_data(data):
    try:
        # Tenta converter data no formato 'yyyy-mm-dd HH:MM:SS'
        return datetime.strptime(data, '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y')
    except ValueError:
        try:
            # Tenta converter data no formato 'dd/mm/yyyy'
            return datetime.strptime(data, '%d/%m/%Y').strftime('%d/%m/%Y')
        except ValueError:
            # Retorna a data original se já estiver no formato correto
            return data

# Conectar ao banco de dados
conn = sqlite3.connect('weather_traffic.db')
cur = conn.cursor()

# Selecionar todos os registros da tabela weather
cur.execute('SELECT weather_id, dt FROM weather')
registros = cur.fetchall()

# Atualizar os registros com datas no formato correto
for weather_id, dt in registros:
    nova_data = formatar_data(dt)
    cur.execute('UPDATE weather SET dt = ? WHERE weather_id = ?', (nova_data, weather_id))

# Commit das mudanças e fechar a conexão
conn.commit()
conn.close()

print("Datas corrigidas para o formato dd/mm/yyyy!")


Datas corrigidas para o formato dd/mm/yyyy!


In [24]:
import sqlite3
import pandas as pd
from tabulate import tabulate

# Conectar ao banco de dados
conn = sqlite3.connect('weather_traffic.db')

# Função para exibir dados de uma tabela
def exibir_dados_tabela(conn, tabela):
    query = f'SELECT * FROM {tabela}'
    df = pd.read_sql_query(query, conn)
    if df.empty:
        print(f'Dados da tabela {tabela}: Vazia')
    else:
        print(f'Dados da tabela {tabela}:')
        print(tabulate(df, headers='keys', tablefmt='pretty'))

# Exibir dados das tabelas
exibir_dados_tabela(conn, 'city')
exibir_dados_tabela(conn, 'weather')
exibir_dados_tabela(conn, 'traffic')

# Fechar a conexão
conn.close()


Dados da tabela city:
+---+---------+-----------+-----------+-----------+
|   | city_id |   name    | coord_lon | coord_lat |
+---+---------+-----------+-----------+-----------+
| 0 |    1    | São Paulo | -46.6361  | -23.5475  |
+---+---------+-----------+-----------+-----------+
Dados da tabela weather:
+---+------------+----------+------------+------------+----------+---------+-----+-----------+-----------------+---------------+---------------+---------------+---------------+------------+----------+-----------+------------+----------+--------+-------------+---------------------+---------------------+------------+-------+-------------+------+
|   | weather_id |   base   | visibility |     dt     | timezone | city_id | cod | main_temp | main_feels_like | main_temp_min | main_temp_max | main_pressure | main_humidity | wind_speed | wind_deg | wind_gust | clouds_all | sys_type | sys_id | sys_country |     sys_sunrise     |     sys_sunset      | id_weather | main  | description | icon |
+

***Integralçao com banco de dados***

In [25]:
import sqlite3

# Função para verificar se a conexão com o banco de dados pode ser estabelecida
def testar_conexao_banco_dados(database_name):
    try:
        conn = sqlite3.connect(database_name)
        conn.close()
        print("Conexão com o banco de dados estabelecida com sucesso.")
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")

# Testar a conexão com o banco de dados
testar_conexao_banco_dados('weather_traffic.db')


Conexão com o banco de dados estabelecida com sucesso.


**Teste de Conexão com o Banco de Dados**

In [26]:
# Função para executar uma consulta de teste no banco de dados
def teste_consulta_banco_dados(database_name):
    try:
        conn = sqlite3.connect(database_name)
        cursor = conn.cursor()
        cursor.execute("SELECT 1")
        result = cursor.fetchone()
        print("Consulta de teste bem-sucedida. Resultado:", result)
        conn.close()
    except Exception as e:
        print(f"Erro ao executar a consulta de teste: {e}")

# Testar a consulta de teste no banco de dados
teste_consulta_banco_dados('weather_traffic.db')


Consulta de teste bem-sucedida. Resultado: (1,)


In [27]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.exc import SQLAlchemyError

Base = declarative_base()

class Traffic(Base):
    __tablename__ = 'traffic'
    id = Column(Integer, primary_key=True)
    route = Column(String)
    distance = Column(Integer)
    duration = Column(Integer)

# Configuração da conexão com o banco de dados
engine = create_engine('sqlite:///weather_traffic.db', echo=True)

# Remover todas as tabelas existentes e recriá-las
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

try:
    # Inserir dados na tabela 'traffic'
    new_traffic = Traffic(route="Main Street", distance=200, duration=300)
    session.add(new_traffic)
    session.commit()
    print("Dados inseridos com sucesso!")
except SQLAlchemyError as e:
    session.rollback()
    print("Erro ao inserir dados:", e)
finally:
    session.close()


2024-06-13 19:02:56,446 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-13 19:02:56,448 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("traffic")
2024-06-13 19:02:56,450 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-13 19:02:56,453 INFO sqlalchemy.engine.Engine 
DROP TABLE traffic
2024-06-13 19:02:56,454 INFO sqlalchemy.engine.Engine [no key 0.00127s] ()
2024-06-13 19:02:56,476 INFO sqlalchemy.engine.Engine COMMIT
2024-06-13 19:02:56,479 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-13 19:02:56,480 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("traffic")
2024-06-13 19:02:56,482 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-13 19:02:56,487 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("traffic")
2024-06-13 19:02:56,490 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-13 19:02:56,511 INFO sqlalchemy.engine.Engine 
CREATE TABLE traffic (
	id INTEGER NOT NULL, 
	route VARCHAR, 
	distance INTEGER, 
	duration INTEGER, 
	PRIMARY KEY (id)
)


202

***Visualização de Dados***

In [28]:
import sqlite3
import pandas as pd

# Função para conectar ao banco de dados e imprimir as colunas de uma tabela
def print_table_columns(db_path, table_name):
    conn = sqlite3.connect(db_path)
    query = f"PRAGMA table_info({table_name})"
    df = pd.read_sql_query(query, conn)
    conn.close()
    print(f"Colunas na tabela {table_name}:")
    print(df[['name', 'type']])

# Caminho para o banco de dados
db_path = '/home/patricia/projeto_Requests/weather_traffic.db'

# Imprime as colunas das tabelas para verificar
print_table_columns(db_path, 'city')
print_table_columns(db_path, 'weather')
print_table_columns(db_path, 'traffic')


Colunas na tabela city:
        name     type
0    city_id  INTEGER
1       name     TEXT
2  coord_lon     REAL
3  coord_lat     REAL
Colunas na tabela weather:
               name     type
0        weather_id  INTEGER
1              base     TEXT
2        visibility  INTEGER
3                dt     TEXT
4          timezone  INTEGER
5           city_id  INTEGER
6               cod  INTEGER
7         main_temp     REAL
8   main_feels_like     REAL
9     main_temp_min     REAL
10    main_temp_max     REAL
11    main_pressure  INTEGER
12    main_humidity  INTEGER
13       wind_speed     REAL
14         wind_deg  INTEGER
15        wind_gust     REAL
16       clouds_all  INTEGER
17         sys_type  INTEGER
18           sys_id  INTEGER
19      sys_country     TEXT
20      sys_sunrise     TEXT
21       sys_sunset     TEXT
22       id_weather  INTEGER
23             main     TEXT
24      description     TEXT
25             icon     TEXT
Colunas na tabela traffic:
       name     type
0       

In [29]:
import sqlite3
import pandas as pd

def query_data():
    conn = sqlite3.connect('/home/patricia/projeto_Requests/weather_traffic.db')
    
    # Verifica se a cidade de São Paulo está no banco de dados
    city_id = pd.read_sql_query("SELECT city_id FROM city WHERE name = 'São Paulo'", conn)
    if city_id.empty:
        print("São Paulo não encontrada no banco de dados.")
        return pd.DataFrame(), pd.DataFrame()

    # Extrai os dados meteorológicos para São Paulo
    df_weather = pd.read_sql_query(f"SELECT * FROM weather WHERE city_id = {city_id['city_id'].iloc[0]}", conn)
    
    # Extrai os dados de tráfego associados às condições meteorológicas em São Paulo
    df_traffic = pd.read_sql_query(f"SELECT * FROM traffic WHERE id IN (SELECT id FROM traffic)", conn)
    
    conn.close()
    return df_weather, df_traffic

# Executa a função para obter os dados
df_weather, df_traffic = query_data()

# Verifica os dados retornados
print("Dados Meteorológicos de São Paulo:", df_weather)
print("Dados de Tráfego de São Paulo:", df_traffic)


Dados Meteorológicos de São Paulo:    weather_id      base  visibility          dt  timezone  city_id  cod  main_temp  main_feels_like  main_temp_min  main_temp_max  main_pressure  main_humidity  wind_speed  wind_deg  wind_gust  clouds_all  sys_type  sys_id sys_country          sys_sunrise           sys_sunset  id_weather   main description icon
0           1  stations       10000  01/06/2023    -10800        1  200     300.23           299.92          299.9         301.09           1021             37        2.68         8       3.13           0         1    8394          BR  2023-06-01 05:30:00  2023-06-01 18:00:00       800.0  Clear   clear sky  01d
1           2  stations       10000  02/06/2023    -10800        1  200     302.15           301.92          300.1         303.09           1020             35        2.90         7       3.50          10         1    8395          BR  2023-06-02 05:30:00  2023-06-02 18:01:00       801.0  Clear  few clouds  02d
2           3  stations   

In [30]:
import sqlite3

# Conectar ao banco de dados correto
conn = sqlite3.connect('/home/patricia/projeto_Requests/weather_traffic.db')
cursor = conn.cursor()

# Listar todas as tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tabelas disponíveis no banco de dados:")
for table in tables:
    print(table[0])

# Verificar se a tabela 'weather' existe e listar dados
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='weather';")
table_exists = cursor.fetchone()

if table_exists:
    print("Tabela 'weather' existe.")
    cursor.execute("SELECT * FROM weather;")
    rows = cursor.fetchall()
    if rows:
        print("Dados na tabela 'weather':")
        for row in rows:
            print(row)
    else:
        print("Tabela 'weather' está vazia.")
else:
    print("Tabela 'weather' não existe.")

conn.close()


Tabelas disponíveis no banco de dados:
weather
city
sqlite_sequence
traffic
Tabela 'weather' existe.
Dados na tabela 'weather':
(1, 'stations', 10000, '01/06/2023', -10800, 1, 200, 300.23, 299.92, 299.9, 301.09, 1021, 37, 2.68, 8, 3.13, 0, 1, 8394, 'BR', '2023-06-01 05:30:00', '2023-06-01 18:00:00', 800, 'Clear', 'clear sky', '01d')
(2, 'stations', 10000, '02/06/2023', -10800, 1, 200, 302.15, 301.92, 300.1, 303.09, 1020, 35, 2.9, 7, 3.5, 10, 1, 8395, 'BR', '2023-06-02 05:30:00', '2023-06-02 18:01:00', 801, 'Clear', 'few clouds', '02d')
(3, 'stations', 10000, '01/06/2024', -10800, 1, 200, 300.23, 299.92, 299.9, 301.09, 1021, 37, 2.68, 8, 3.13, 0, 1, 8394, 'BR', '01/06/2024', '01/06/2024', None, 'Clear', 'clear sky', '01d')
(4, 'stations', 10000, '02/06/2024', -10800, 1, 200, 300.23, 299.92, 299.9, 301.09, 1021, 37, 2.68, 8, 3.13, 0, 1, 8394, 'BR', '02/06/2024', '02/06/2024', None, 'Clear', 'clear sky', '01d')
(5, 'stations', 10000, '03/06/2024', -10800, 1, 200, 300.23, 299.92, 299.9, 30

In [35]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import sqlite3

# Função para consultar dados
def query_data():
    conn = sqlite3.connect('/home/patricia/projeto_Requests/weather_traffic.db')
    df_weather = pd.read_sql_query("SELECT dt, main_temp, sys_sunrise, sys_sunset FROM weather WHERE city_id = 1", conn)
    conn.close()
    return df_weather

# Função para formatar a data
def formatar_data(df):
    df['dt'] = pd.to_datetime(df['dt'], errors='coerce').dt.strftime('%d/%m/%Y')
    return df

# Obter os dados
df_weather = query_data()
df_weather = formatar_data(df_weather)
df_weather['main_temp'] = df_weather['main_temp'] - 273.15  # Convertendo de Kelvin para Celsius

# Calcular média e desvio padrão da temperatura
media_temp = df_weather['main_temp'].mean()
desvio_padrao_temp = df_weather['main_temp'].std()

# Inicializar o aplicativo Dash
app = dash.Dash(__name__)

# Layout do aplicativo com estilo CSS
app.layout = html.Div(
    style={'backgroundColor': '#f9f9f9', 'fontFamily': 'Arial, sans-serif', 'padding': '20px'},
    children=[
        html.H1(
            "Dashboard Meteorológico de São Paulo",
            style={'textAlign': 'center', 'color': '#333', 'marginBottom': '40px'}
        ),
        dcc.Graph(id='temp-graph', style={'height': '70vh'}),
        html.Div(
            [
                html.P(f"Média da Temperatura: {media_temp:.2f} °C"),
                html.P(f"Desvio Padrão da Temperatura: {desvio_padrao_temp:.2f} °C"),
                html.P(f"Nascer do Sol: {df_weather['sys_sunrise'].iloc[0]}"),
                html.P(f"Pôr do Sol: {df_weather['sys_sunset'].iloc[0]}")
            ],
            style={
                'textAlign': 'center',
                'fontSize': '20px',
                'color': '#666',
                'padding': '10px',
                'backgroundColor': 'wheat',
                'border': '2px solid black',
                'borderRadius': '5px',
                'display': 'inline-block',
                'marginTop': '20px'
            }
        )
    ]
)

# Callback para atualizar o gráfico
@app.callback(
    Output('temp-graph', 'figure'),
    [Input('temp-graph', 'id')]
)
def update_graph(_):
    if df_weather.empty:
        fig = {}
    else:
        fig = px.line(
            df_weather,
            x='dt',
            y='main_temp',
            title='Temperatura ao longo do Tempo',
            labels={'dt': 'Data e Hora', 'main_temp': 'Temperatura (°C)'},
            template='plotly_white'
        )
        fig.add_traces([
            dict(
                type="scatter",
                x=df_weather['dt'],
                y=df_weather['main_temp'],
                fill='tozeroy',
                mode='lines',
                line=dict(color='lightblue')
            )
        ])
        fig.update_layout(
            title={'font': {'size': 24}, 'x': 0.5, 'xanchor': 'center'},
            xaxis_title={'font': {'size': 18}},
            yaxis_title={'font': {'size': 18}},
            margin={'l': 40, 'r': 40, 't': 40, 'b': 40}
        )
    
    return fig

# Executar o aplicativo
if __name__ == '__main__':
    app.run_server(debug=True)
